In [2]:
# Random Forest Classification

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio import SeqUtils

# Importing the dataset
dataset = pd.read_csv('final file hopefully no errors.csv')

#for fib2.cvs only

dataset.loc[dataset['Gen'] == 'ssRNA(+)', 'Gen'] = 1
dataset.loc[dataset['Gen'] == 'ssRNA(-)', 'Gen'] = 0

p = dataset.loc[dataset['Gen'] == 1].reset_index()
n = dataset.loc[dataset['Gen'] == 0].reset_index()

pn = pd.concat([p,n]).reset_index(drop = True)
res = pd.read_csv('res 340.csv')
r = res[['ATC', 'ATCA', 'ATCT', 'TGGT']].copy()
pnr = pd.concat([pn, r], axis = 1)
pnr = pnr.drop(index = pnr.loc[pnr['MolTyp'] == 'DNA'].index)
pnr = pnr.drop(columns=['index', 'Id', 'MolTyp', 'Topol', 'Des'])
pnr = pnr.rename(columns={'ATC' : 'E1', 'ATCA' : 'E2', 'ATCT' : 'E3', 'TGGT' : 'E4'})

def orf_translator(record, table, min_pro_len):
    s1 = []
    s2 = []
    for strand, nuc in [(+1, record.seq), (-1, record.seq.reverse_complement())]:
        for frame in range(3):
            length = 3 * ((len(record)-frame) // 3) #Multiple of three
            for pro in nuc[frame:frame+length].translate(table).split("*"):
                if len(pro) >= min_pro_len:
                    if strand == 1:
                        s1.append(pro)
                    else:
                        s2.append(pro)
    return s1,s2


def whole(pnr):

    ps = pnr.loc[pnr['Gen'] == 1].sample(200)
    ns = pnr.loc[pnr['Gen'] == 0].sample(200)

    pnrs = pd.concat([ps,ns])
    
    
    #feature 1: length
    pnrs['F1'] = [len(i) for i in pnrs['Seq'].to_list()]
    
    f2, f3, f4, f5, f6, f7 = [],[],[],[],[],[]

    for i in pnrs['Seq'].to_list():
        a = SeqRecord(Seq(i))
        (s1, s2) = orf_translator(a, 11, 15)
        f2.append(len(s1))
        f3.append(len(s2))

    pnrs['F2'] = f2
    pnrs['F3'] = f3
    
    #Splitting features and labels
    X = pnrs.iloc[:, 3:].values.astype(float)
    y = pnrs.iloc[:, 1].values.astype(float)
    
    # Splitting the dataset into the Training set and Test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
    
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)

    return cm  

In [10]:
for i in range(100):
    cm = whole(pnr)
    print((cm[0][0] + cm[1][1]) / (cm.sum()))

0.96
0.99
0.96
0.96
0.95
0.96
0.96
0.96
0.94
0.97
0.96
0.95
0.96
0.95
0.95
0.94
0.95
0.97
0.95
0.93
0.94
0.96
0.96
0.98
0.94
0.95
0.96
1.0
0.98
0.97
0.93
0.96
0.95
0.95
0.95
0.97
0.96
0.99
0.95
0.94
0.94
0.97
0.96
0.96
0.95
0.96
0.97
0.98
0.94
0.96
0.98
0.95
0.98
0.95
0.97
0.96
0.97
0.94
0.93
0.92
0.96
0.92
0.96
0.97
0.94
0.96
0.95
0.97
0.94
0.94
0.93
0.95
0.98
0.97
0.95
0.95
0.94
0.96
0.97
0.94
0.94
0.94
0.93
0.95
0.96
0.94
0.92
0.97
0.94
0.95
0.98
0.96
0.94
0.94
0.95
0.97
0.92
0.93
0.92
0.97
